In [1]:
# Importing pandas library
import pandas as pd
import os

In [2]:
WEO_file = os.path.join('Resources','WEOApr2021all.xls')
df = pd.read_csv(WEO_file, sep = '\t', engine = 'python')
#df

In [3]:
#Pandas copy rows containing 'WEO Subject Code' in 'NGDPD','LUR','LE','LP' GDP, Unemployment, Employment,Total Population and drop columns
WEO_df = df[df['WEO Subject Code'].isin(['NGDPD','LUR','LE','LP'])].drop(columns={'WEO Country Code','Country','Subject Descriptor','Subject Notes','Units','Scale','Country/Series-specific Notes','Estimates Start After'}).reset_index(drop=True)
WEO_df

,ISO,WEO Subject Code,1980,1981,1982,1983,1984,1985,1986,1987,...,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026
0,AFG,NGDPD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.91,18.401,18.876,19.132,19.938,21.201,22.01,22.754,23.896,24.553
1,AFG,LUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,LE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,LP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29.7,31.6,32.2,32.941,33.698,34.473,35.266,36.077,36.907,37.756
4,ALB,NGDPD,1.946,2.229,2.296,2.319,2.29,2.339,2.587,2.566,...,13.053,15.147,15.276,15.147,17.138,18.32,19.662,21.009,22.422,23.903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,ZMB,LP,5.907,6.101,6.297,6.495,6.695,6.897,7.104,7.317,...,17.238,17.773,18.321,18.882,19.456,20.043,20.643,21.259,21.892,22.543
776,ZWE,NGDPD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21.89,21.093,19.273,21.038,26.085,29.363,29.319,29.979,30.88,32.042
777,ZWE,LUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
778,ZWE,LE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# used melt to convert columns of dates to "Year" row with data values
WEO_df.rename(columns = {'ISO':'NOC', 'WEO Subject Code': 'SubjectCode'}, inplace = True)
WEO_df = WEO_df.melt(id_vars=['NOC','SubjectCode'],
        var_name="Year", 
        value_name="Value")
WEO_df

,NOC,SubjectCode,Year,Value
0,AFG,NGDPD,1980,NaN
1,AFG,LUR,1980,NaN
2,AFG,LE,1980,NaN
3,AFG,LP,1980,NaN
4,ALB,NGDPD,1980,1.946
...,...,...,...,...
36655,ZMB,LP,2026,22.543
36656,ZWE,NGDPD,2026,32.042
36657,ZWE,LUR,2026,NaN
36658,ZWE,LE,2026,NaN


In [5]:
# Reset DataFrame with columns in desired order
WEO_df = WEO_df[['NOC','Year','SubjectCode','Value']]
WEO_df = WEO_df.rename({'SubjectCode': 'SubjectCode'},axis=1)
WEO_df

,NOC,Year,SubjectCode,Value
0,AFG,1980,NGDPD,NaN
1,AFG,1980,LUR,NaN
2,AFG,1980,LE,NaN
3,AFG,1980,LP,NaN
4,ALB,1980,NGDPD,1.946
...,...,...,...,...
36655,ZMB,2026,LP,22.543
36656,ZWE,2026,NGDPD,32.042
36657,ZWE,2026,LUR,NaN
36658,ZWE,2026,LE,NaN


In [6]:
#Use Pivote to convert SubjectCode row values into columns, and then name them appropriately
test = WEO_df.pivot(index=['NOC','Year'],
                    columns='SubjectCode',
                    values='Value').reset_index()
test.fillna(0, inplace=True)
WEO_df = test.rename({'LE': 'Employment', 'LP': 'Population', 'LUR': 'UnemploymentRate', 'NGDPD': 'GDP'}, axis=1) 
WEO_df[WEO_df.NOC.isin(['USA'])]
#GDP, current prices	U.S. dollars	Billions
#Unemployment rate	Percent of total labor force	
#Employment	Persons	Millions
#Population	Persons	Millions

SubjectCode,NOC,Year,Employment,Population,UnemploymentRate,GDP
8554,USA,1980,99.303,227.622,7.175,"2,857.33"
8555,USA,1981,100.4,229.916,7.617,"3,207.03"
8556,USA,1982,99.529,232.128,9.708,"3,343.80"
8557,USA,1983,100.822,234.247,9.6,"3,634.03"
8558,USA,1984,105.003,236.307,7.508,"4,037.65"
8559,USA,1985,107.154,238.416,7.192,"4,339.00"
8560,USA,1986,109.601,240.593,7,"4,579.63"
8561,USA,1987,112.439,242.751,6.175,"4,855.25"
8562,USA,1988,114.974,244.968,5.492,"5,236.43"
8563,USA,1989,117.327,247.286,5.258,"5,641.60"


In [7]:
WEO_df.dtypes

SubjectCode
NOC                 object
Year                object
Employment          object
Population          object
UnemploymentRate    object
GDP                 object
dtype: object

In [8]:
#removing commas from GDP, replace country codes as needed to fit athlete data ISO --> NOC
WEO_df['GDP'] = WEO_df['GDP'].str.replace(',', '')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('DZA','ALG')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('AGO','ANG')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('ATG','ANT')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('ABW','ARU')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('BGD','BAN')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('BRB','BAR')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('BLZ','BIZ')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('BTN','BHU')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('BWA','BOT')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('BGR','BUL')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('BFA','BUR')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('KHM','CAM')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('TCD','CHA')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('CHL','CHI')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('CRI','CRC')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('HRV','CRO')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('DNK','DEN')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('SLV','ESA')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('GNQ','GEQ')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('FJI','FIJ')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('DEU','GER')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('GRC','GRE')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('GRD','GRN')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('GTM','GUA')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('GIN','GUI')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('HTI','HAI')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('HND','HON')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('IDN','INA')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('UVK','KOS')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('KWT','KUW')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('LVA','LAT')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('LBN','LIB')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('LSO','LES')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('LBY','LBA')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('MDG','MAD')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('MWI','MAW')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('MYS','MAS')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('MRT','MTN')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('MUS','MRI')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('MNG','MGL')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('MMR','MYA')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('NPL','NEP')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('NLD','NED')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('NIC','NCA')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('NER','NIG')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('NGA','NGR')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('OMN','OMA')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('PRY','PAR')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('PHL','PHI')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('PRT','POR')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('PRI','PUR')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('WSM','SAM')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('SAU','KSA')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('SYC','SEY')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('SVN','SLO')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('SLB','SOL')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('ZAF','RSA')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('LKA','SRI')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('SDN','SUD')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('CHE','SUI')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('TZA','TAN')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('TGO','TOG')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('TON','TGA')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('ARE','UAE')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('URY','URU')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('VUT','VAN')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('VNM','VIE')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('ZMB','ZAM')
WEO_df['NOC'] = WEO_df['NOC'].str.replace('ZWE','ZIM')

WEO_df['NOC'] = WEO_df['NOC'].str.replace('IRN','IRI')

#must be done in order
WEO_df['NOC'] = WEO_df['NOC'].str.replace('BRN','BRU') #Brunei Darussalam ISO --> Brunei NOC
WEO_df['NOC'] = WEO_df['NOC'].str.replace('BHR','BRN') #Bahrain ISO --> Bahrain NOC




In [9]:
#need the Year to match the Year in athlete_file in order to merge
WEO_df['Year'] = WEO_df['Year'].astype(str).astype(float)

In [10]:
athlete_file = os.path.join("Resources","cleaned_athlete_data.csv")
athlete_data_df = pd.read_csv(athlete_file)
athlete_data_df.dtypes

ID              int64
Name           object
Sex            object
Age           float64
Height        float64
Weight        float64
Team           object
NOC            object
Games          object
Year            int64
Season         object
City           object
Sport          object
Event          object
Medal_Type     object
Medal          object
Row             int64
dtype: object

In [11]:
athlete_data_df_gdp = pd.merge(athlete_data_df, WEO_df, on=['NOC','Year'], how='left')
#athlete_data_df_gdp


In [12]:
#athlete_data_df_gdp.isnull().sum()
#athlete_data_df_gdp(athlete_data_df_gdp['Employment']).isnull,'NOC')
#athlete_data_df_gdp[athlete_data_df_gdp["Year"] > 2000 & athlete_data_df_gdp["Employment"].isnull()]

In [13]:
athlete_data_df_gdp.to_csv('Resources/cleaned_athlete_gdp_data.csv', index=False)

In [14]:
missing=athlete_data_df_gdp.loc[athlete_data_df_gdp["Employment"].isnull()]
missing.to_csv('Resources/missing.csv', index=False)

In [15]:
#count of how many Nulls (data that might have to be eliminated in order to use GDP on atheletes that have that data)
athlete_data_df_gdp.isnull().sum().sort_values(ascending = False)

GDP                 52160
UnemploymentRate    51342
Population          51342
Employment          51342
City                    0
Row                     0
Medal                   0
Medal_Type              0
Event                   0
Sport                   0
ID                      0
Name                    0
Year                    0
Games                   0
NOC                     0
Team                    0
Weight                  0
Height                  0
Age                     0
Sex                     0
Season                  0
dtype: int64